# Fast Pole Angle Detection
## Optimized for Low Latency

In [ ]:
import numpy as np
import cv2
from picamera2 import Picamera2
import time
from IPython.display import display, Image, clear_output

print("Libraries imported!")

In [ ]:
# Initialize camera
picam2 = Picamera2()
config = picam2.create_preview_configuration(
    main={"format": "RGB888", "size": (1280, 720)}
)
picam2.configure(config)
picam2.start()
time.sleep(0.5)

print("Camera ready!")

In [ ]:
# Detection parameters
lower_green = np.array([35, 50, 50])
upper_green = np.array([85, 255, 255])
min_area = 100

# Pre-create kernel for morphology (reuse)
kernel = np.ones((5, 5), np.uint8)

print("Parameters configured")

## Fast Live Angle Detection

Optimized for minimal latency. Press STOP to interrupt.

In [ ]:
print("Starting fast live detection...\n")

try:
    frame_count = 0
    
    while True:
        # Capture
        frame = picam2.capture_array()
        
        # Green detection (minimal processing)
        hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(hsv, lower_green, upper_green)
        
        # Single morph operation instead of two
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Get centroids (fast)
        centroids = []
        for cnt in contours:
            if cv2.contourArea(cnt) > min_area:
                M = cv2.moments(cnt)
                if M["m00"] != 0:
                    centroids.append((int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])))
        
        # Draw (minimal operations)
        if len(centroids) == 2:
            p1, p2 = centroids
            
            # Calculate angle
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            angle = abs(np.pi/2 - np.arctan2(dy, dx))
            
            # Draw line
            cv2.line(frame, p1, p2, (0, 255, 0), 2)
            
            # Draw markers (simple circles)
            cv2.circle(frame, p1, 8, (255, 0, 0), -1)
            cv2.circle(frame, p2, 8, (255, 0, 0), -1)
            
            # Minimal text
            text = f"{angle:.3f} rad ({np.degrees(angle):.1f}deg)"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Just show marker count
            cv2.putText(frame, f"Markers: {len(centroids)}/2", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Encode and display (minimal quality for speed)
        _, buffer = cv2.imencode('.jpg', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, 85])
        clear_output(wait=True)
        display(Image(data=buffer.tobytes()))
        
        frame_count += 1
        
        # No sleep - run as fast as possible
        
except KeyboardInterrupt:
    print(f"\nStopped. Processed {frame_count} frames")

In [ ]:
picam2.stop()
print("Camera stopped")